In [2]:
import os
import pandas as pd
import random
import sqlalchemy
from sqlalchemy import create_engine, exc
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Database configuration from .env file
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")

# Create SQLAlchemy engine
engine = create_engine(f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}")

# Load and populate movies table from CSV
def populate_movies_table(csv_file_path):
    try:
        print("Loading CSV file...")
        movies_df = pd.read_csv(csv_file_path)
        print("CSV file loaded successfully. Populating movies table...")

        movies_df = movies_df.drop_duplicates(subset='id')
        # Rename columns to match the database table
        movies_df.rename(columns={
            'id': 'tmdb_movie_id',
            # Add other necessary column renamings here
        }, inplace=True)

        movies_df.to_sql('movies', engine, if_exists='append', index=False)
        print("Movies table populated successfully.")
    except pd.errors.ParserError:
        print("Error: Failed to parse CSV file.")
    except sqlalchemy.exc.SQLAlchemyError as e:
        print(f"SQLAlchemy Error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# Function to create users
def create_users(num_users):
    try:
        for i in range(num_users):
            # Generate user details
            username = f"user{i}"
            email = f"user{i}@example.com"
            password_hash = "some_hash"  # Generate a secure hash in practice
            
            # Insert user into database
            with engine.connect() as connection:
                connection.execute(
                    "INSERT INTO users (username, email, password_hash) VALUES (%s, %s, %s)",
                    (username, email, password_hash)
                )
        print(f"{num_users} users created successfully.")
    except sqlalchemy.exc.SQLAlchemyError as e:
        print(f"SQLAlchemy Error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# Function to create watchlists and assign movies
def create_watchlists_and_movies(num_users):
    # Define directors and genres
    directors = ["Steven Spielberg", "Martin Scorsese"]
    genres = ["Drama", "Horror", "Action/Adventure"]

    try:
        for user_id in range(1, num_users + 1):
            # Create watchlists for each user
            with engine.connect() as connection:
                # Watched watchlist
                connection.execute("INSERT INTO watchlists (user_id, name) VALUES (%s, 'Watched')", (user_id,))
                watched_id = connection.execute("SELECT LAST_INSERT_ID()").fetchone()[0]

                # Additional watchlists for five users
                if user_id <= 5:
                    for name in directors + genres:
                        connection.execute("INSERT INTO watchlists (user_id, name) VALUES (%s, %s)", (user_id, name))

            # Populate Watched watchlist
            with engine.connect() as connection:
                # Select movies based on criteria
                # Add your logic here to select 45 films from two random genres, 25 films from directors, and 30 random movies

                # Example: Inserting random movies into Watched watchlist
                movie_ids = connection.execute("SELECT tmdb_movie_id FROM movies ORDER BY RAND() LIMIT 100").fetchall()
                for movie_id in movie_ids:
                    connection.execute("INSERT INTO watchlist_movies (watchlist_id, tmdb_movie_id) VALUES (%s, %s)", (watched_id, movie_id[0]))

            # Additional logic to populate specific director and genre watchlists for first five users

        print("Watchlists and movies assigned successfully.")
    except sqlalchemy.exc.SQLAlchemyError as e:
        print(f"SQLAlchemy Error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# Function to generate ratings
def generate_ratings(num_users):
    try:
        for user_id in range(1, num_users + 1):
            with engine.connect() as connection:
                watched_movies = connection.execute("SELECT tmdb_movie_id FROM watchlist_movies WHERE watchlist_id = (SELECT watchlist_id FROM watchlists WHERE user_id = %s AND name = 'Watched')", (user_id,)).fetchall()
                for movie in watched_movies:
                    # Fetch original movie rating
                    original_rating = connection.execute("SELECT vote_average FROM movies WHERE tmdb_movie_id = %s", (movie[0],)).fetchone()[0]
                    # Convert to 0-5 scale with variance
                    new_rating = round((original_rating / 2) + random.uniform(-0.5, 0.5), 1)
                    new_rating = max(0, min(5, new_rating)) 

                    # Insert rating into database
                    connection.execute("INSERT INTO ratings (user_id, tmdb_movie_id, rating) VALUES (%s, %s, %s)", (user_id, movie[0], new_rating))

        print("Ratings generated successfully.")
    except sqlalchemy.exc.SQLAlchemyError as e:
        print(f"SQLAlchemy Error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

if __name__ == "__main__":
    csv_file_path = '../data/TMDB_movie_dataset_v11.csv'
    populate_movies_table(csv_file_path)

    num_users = 15
    create_users(num_users)
    create_watchlists_and_movies(num_users)
    generate_ratings(num_users)

Loading CSV file...
CSV file loaded successfully. Populating movies table...
SQLAlchemy Error: (pymysql.err.IntegrityError) (1062, "Duplicate entry '27205' for key 'movies.PRIMARY'")
[SQL: INSERT INTO movies (tmdb_movie_id, title, vote_average, vote_count, status, release_date, revenue, runtime, adult, backdrop_path, budget, homepage, imdb_id, original_language, original_title, overview, popularity, poster_path, tagline, genres, production_companies, production_countries, spoken_languages) VALUES (%(tmdb_movie_id)s, %(title)s, %(vote_average)s, %(vote_count)s, %(status)s, %(release_date)s, %(revenue)s, %(runtime)s, %(adult)s, %(backdrop_path)s, %(budget)s, %(homepage)s, %(imdb_id)s, %(original_language)s, %(original_title)s, %(overview)s, %(popularity)s, %(poster_path)s, %(tagline)s, %(genres)s, %(production_companies)s, %(production_countries)s, %(spoken_languages)s)]
[parameters: [{'tmdb_movie_id': 27205, 'title': 'Inception', 'vote_average': 8.364, 'vote_count': 34495, 'status': 'Re